In [239]:
import pandas as pd
from tkinter import Tk, filedialog
import os

# Function to open file dialog and select files
def upload_file(label):
    # Create a hidden Tkinter window (for file dialog)
    root = Tk()
    root.withdraw()  # Hide the root window
    root.call('wm', 'attributes', '.', '-topmost', True)  # Bring file dialog to the front
    
    # Open file dialog
    file_path = filedialog.askopenfilename(
        title=f'Select {label} file',
        filetypes=[('Excel files', '*.xlsx')]
    )
    
    # Check if a file was selected
    if file_path:
        print(f'{label} file selected: {file_path}')
        return file_path
    else:
        print(f'No {label} file selected.')
        return None

# Manually input file paths for BOM A and BOM B using file dialog
print('Please select First BoM file:')
bom_a_file = upload_file('BOM A')

print('Please select Second BoM file:')
bom_b_file = upload_file('BOM B')

Please select First BoM file:
BOM A file selected: C:/Users/diegobecerra/Downloads/Tacton BOM Plain Test.xlsx
Please select Second BoM file:
BOM B file selected: C:/Users/diegobecerra/Downloads/SP-BOM test.xlsx


In [293]:
bom_a=pd.read_excel(bom_a_file,header=None) #We read the file eliminating blank spaces
bom_a_headers= bom_a.apply(lambda x: x.notna().all(), axis=1).idxmax() #obtaing the first non empty row in order to get the headers
bom_a=pd.read_excel(bom_a_file, header=bom_a_headers).dropna(how='all') #Obtaing the files with the headers as we extracted before

#Same procedure done for bom a now for bom b
bom_b=pd.read_excel(bom_b_file,header=None)
bom_b_headers= bom_b.apply(lambda x: x.notna().all(), axis=1).idxmax() 
bom_b=pd.read_excel(bom_b_file, header=bom_b_headers).dropna(how='all') 

#asking infomration to ensure correct proccessing

article_code_col_a = 'Article Code' #input('Enter the column name for Article Code in BOM A: ') #This is just in ase of wanting manual input 
quantity_col_a = 'Qty' #input('Enter the column name for Quantity in BOM A: ')
article_code_col_b = 'Article Code' #input('Enter the column name for Article Code in BOM B: ')
quantity_col_b = 'Qty' #input('Enter the column name for Quantity in BOM B: ')

# Setting the indices for comparison
bom_a.set_index(article_code_col_a, inplace=True)
bom_b.set_index(article_code_col_b, inplace=True)
 

In [295]:
#Viewing the informtion of BOM a
file_names='File {} contains the following information'
print(file_names.format(os.path.basename(bom_a_file)))
bom_a=bom_a.sort_index(ascending=True)
display(bom_a)

File Tacton BOM Plain Test.xlsx contains the following information


,Qty,Description
Article Code,,
100934.0,1.0,Crone Boiler CLW350
102210.0,2.0,Steel welding elbow 90°ø139.7
102215.0,6.0,Bend 3S-90 ø323.9
102215.0,9.0,Steel welding elbow 90° ø323.9
102238.0,3.0,Steel welding reducer ø323 x 219.1
103567.0,2.0,Steel weldneck flange ø139.7 PN6
103671.0,3.0,Carbon Steel Pipe ø139.7 mm x4 mm 6 mts (Stuk)
103675.0,8.0,Carbon Steel Pipe ø323.9 mm x4 mm 6 mts (Stuk)
103675.0,4.0,Carbon Steel Pipe ø323.9 mm x4 mm 6 mts (Stuk)


In [297]:
#viewing the information from BOM b
print(file_names.format(os.path.basename(bom_b_file)))
bom_b=bom_b.sort_index(ascending=True)
display(bom_b)

File SP-BOM test.xlsx contains the following information


,Qty,Description
Article Code,,
100584.0,2.0,Steel end plate ø323
102212.0,3.0,Steel welding elbow 90° ø168.3
102215.0,13.0,Steel welding elbow 90° ø323.9
102238.0,3.0,Steel welding reducer ø323 x 219.1
103567.0,2.0,Steel weldneck flange ø168.3 PN6
103671.0,2.0,Steel pipe ø139.7
103675.0,4.0,Steel pipe ø323.9
103741.0,2.0,Gasket ø150 PN6
104069.0,1.0,-BUNDLE-Butterfly valve DN300 PN6 + actuator


In [397]:
# Comparison
missing_in_b = bom_a[~bom_a.index.isin(bom_b.index)]
missing_in_a = bom_b[~bom_b.index.isin(bom_a.index)]
quantity_diff = bom_a.merge(bom_b,left_index=True,right_index=True,suffixes=('_A', '_B'))
quantity_diff['Quantity Difference'] = quantity_diff['Qty_A'] - quantity_diff['Qty_B']

quantity_diff


,Qty_A,Description_A,Qty_B,Description_B,Quantity Difference
Article Code,,,,,
102215.0,6.0,Bend 3S-90 ø323.9,13.0,Steel welding elbow 90° ø323.9,-7.0
102215.0,9.0,Steel welding elbow 90° ø323.9,13.0,Steel welding elbow 90° ø323.9,-4.0
102238.0,3.0,Steel welding reducer ø323 x 219.1,3.0,Steel welding reducer ø323 x 219.1,0.0
103567.0,2.0,Steel weldneck flange ø139.7 PN6,2.0,Steel weldneck flange ø168.3 PN6,0.0
103671.0,3.0,Carbon Steel Pipe ø139.7 mm x4 mm 6 mts (Stuk),2.0,Steel pipe ø139.7,1.0
103675.0,8.0,Carbon Steel Pipe ø323.9 mm x4 mm 6 mts (Stuk),4.0,Steel pipe ø323.9,4.0
103675.0,4.0,Carbon Steel Pipe ø323.9 mm x4 mm 6 mts (Stuk),4.0,Steel pipe ø323.9,0.0
104362.0,1.0,-BUNDLE-Butterfly valve DN150 PN6,1.0,-BUNDLE-Butterfly valve DN150 PN6,0.0
183862.0,1.0,-BUNDLE-Gate valve Econ DN150 PN10 Cast Iron,1.0,-BUNDLE-Gate valve Econ DN150 PN10 Cast Iron,0.0


In [399]:
display(missing_in_b)

,Qty,Description
Article Code,,
100934.0,1.0,Crone Boiler CLW350
102210.0,2.0,Steel welding elbow 90°ø139.7
104539.0,2.0,Spring Safety DN125
187280.0,1.0,-BUNDLE-Butterfly valve DN125 PN6
187284.0,2.0,-BUNDLE-Butterfly valve DN300 PN6
187303.0,2.0,"BUNDLE-Spirotop Air Release Automatic 1/2"""
187318.0,1.0,BUNDLE-Butterfly valve DN300 PN6 + actuator 240V Sun Yeh
187360.0,1.0,-BUNDLE-Check DN300
187428.0,1.0,"-BUNDLE-Fill-drain Valve 1"""


In [401]:
display(missing_in_a)

,Qty,Description
Article Code,,
100584.0,2.0,Steel end plate ø323
102212.0,3.0,Steel welding elbow 90° ø168.3
103741.0,2.0,Gasket ø150 PN6
104069.0,1.0,-BUNDLE-Butterfly valve DN300 PN6 + actuator
104365.0,3.0,-BUNDLE-Butterfly valve DN300 PN6
104737.0,1.0,-BUNDLE-3 Way valve Centra DN200 PN6
106338.0,8.0,Bolt galvanized m16x110


In [403]:
#Exporting to a Excel File
writer = pd.ExcelWriter('Comparison.xlsx', engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
quantity_diff.to_excel(writer, sheet_name='Quantities')
missing_in_b.to_excel(writer,sheet_name='MissinginSP')
missing_in_a.to_excel(writer,sheet_name='MissinginTacton')
# Close the Pandas Excel writer and output the Excel file.
writer.close()